In [ ]:
!pip install transformers >> /dev/null

In [ ]:
!pip install sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00


In [ ]:
import torch
from typing import List
import numpy as np
import pandas as pd
from collections import Counter
from torch import nn
import torch.nn.functional as F
from transformers import AutoTokenizer, TFAutoModel
import os, shutil
from pathlib import Path
import tensorflow as tf

In [ ]:
!pip freeze | grep tensorflow

tensorflow==2.12.0
tensorflow-datasets==4.8.3
tensorflow-estimator==2.12.0
tensorflow-gcs-config==2.12.0
tensorflow-hub==0.13.0
tensorflow-io-gcs-filesystem==0.32.0
tensorflow-metadata==1.13.0
tensorflow-probability==0.19.0


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!which nvcc

/usr/local/cuda/bin/nvcc


In [ ]:
!ls /usr/local/

bin    cuda-11	  games		      include  licensing  share
colab  cuda-11.8  _gcs_config_ops.so  lib      man	  src
cuda   etc	  images	      lib64    sbin


In [ ]:
!nvidia-smi

Fri Apr  7 21:39:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
tf.__version__

'2.12.0'

In [ ]:
# Download coach repositories dataset
!gdown 1K_JZc38HNP9N4JpTwssry5HBps7KOuST

Downloading...
From: https://drive.google.com/uc?id=1K_JZc38HNP9N4JpTwssry5HBps7KOuST
To: /content/coach_repos_zip.zip
100% 294M/294M [00:07<00:00, 36.9MB/s]


In [ ]:
!unzip coach_repos_zip.zip

Archive:  coach_repos_zip.zip
   creating: coach_repos_zip/
  inflating: coach_repos_zip/zanavi-master.zip  
  inflating: __MACOSX/coach_repos_zip/._zanavi-master.zip  
  inflating: coach_repos_zip/compass-keyboard-master.zip  
  inflating: __MACOSX/coach_repos_zip/._compass-keyboard-master.zip  
  inflating: coach_repos_zip/BipolAlarm-master.zip  
  inflating: __MACOSX/coach_repos_zip/._BipolAlarm-master.zip  
  inflating: coach_repos_zip/cat-avatar-generator-app-master.zip  
  inflating: __MACOSX/coach_repos_zip/._cat-avatar-generator-app-master.zip  
  inflating: coach_repos_zip/Android-PreferencesManager-master.zip  
  inflating: __MACOSX/coach_repos_zip/._Android-PreferencesManager-master.zip  
  inflating: coach_repos_zip/android-binaural-beats-master.zip  
  inflating: __MACOSX/coach_repos_zip/._android-binaural-beats-master.zip  
  inflating: coach_repos_zip/android-xbmcremote-master.zip  
  inflating: __MACOSX/coach_repos_zip/._android-xbmcremote-master.zip  
  inflating: coac

In [ ]:
# !rm -rf ./coach_repos_zip

In [ ]:
# UNZIP files RECURSIVELY
!find ./coach_repos_zip -name "*.zip" | while read filename; do unzip -o -d "`dirname "$filename"`" "$filename"; done;

Streaming output truncated to the last 5000 lines.
  inflating: ./coach_repos_zip/zanavi-master/configure.in  
   creating: ./coach_repos_zip/zanavi-master/contrib/
  inflating: ./coach_repos_zip/zanavi-master/contrib/android_build.sh  
  inflating: ./coach_repos_zip/zanavi-master/depcomp  
  inflating: ./coach_repos_zip/zanavi-master/download-androidstudio-files.bat  
  inflating: ./coach_repos_zip/zanavi-master/download-androidstudio-files.sh  
  inflating: ./coach_repos_zip/zanavi-master/install-sh  
   creating: ./coach_repos_zip/zanavi-master/intl/
  inflating: ./coach_repos_zip/zanavi-master/intl/CMakeLists.txt  
 extracting: ./coach_repos_zip/zanavi-master/intl/ChangeLog  
  inflating: ./coach_repos_zip/zanavi-master/intl/Makefile.in  
  inflating: ./coach_repos_zip/zanavi-master/intl/VERSION  
  inflating: ./coach_repos_zip/zanavi-master/intl/bindtextdom.c  
  inflating: ./coach_repos_zip/zanavi-master/intl/config.charset  
  inflating: ./coach_repos_zip/zanavi-master/intl/dcge

In [ ]:
# Deleting __MACOSX directories 
!rm -r __MACOSX
!rm -r coach_repos_zip/__MACOSX

!ls coach_repos_zip | grep MACOS

In [ ]:
# Deleting .zip recursively after they have been extracted
!find ./coach_repos_zip -name "*.zip" -type f -delete

In [ ]:
!rm -rf coach_repos_zip/archi-master
!rm -rf coach_repos_zip/archi-master-master

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# BigBirdForMaskedLM]  # Does not give last hidden state

from transformers import AutoTokenizer, BigBirdModel
from transformers import BigBirdForMaskedLM, BigBirdTokenizer, BigBirdConfig, DataCollatorForLanguageModeling
import torch

output_dir = "/content/gdrive/MyDrive/bigbirdoutput"
model_name = "google/bigbird-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BigBirdModel.from_pretrained(output_dir).to(device)

Some weights of the model checkpoint at /content/gdrive/MyDrive/bigbirdoutput were not used when initializing BigBirdModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
PAD = tokenizer.pad_token
CLS = tokenizer.cls_token
SEP = tokenizer.sep_token
MAX_LEN = tokenizer.model_max_length
BASE = Path('./coach_repos_zip')
assert MAX_LEN == 4096

In [ ]:
def pad_max_len(tok_ids: List[int], max_len = MAX_LEN) -> List[int]:
    """Pad sequence to max length i.e., 512"""
    pad_len = max_len - len(tok_ids) 
    padding = [tokenizer.convert_tokens_to_ids(PAD)] * pad_len
    return tok_ids + padding

def get_input_mask(toks_padded: List[int]):
    """Calculate attention mask
     - 1 for tokens that are **not masked**,
     - 0 for tokens that are **masked**."""

    return np.where(np.array(toks_padded) == tokenizer.convert_tokens_to_ids(PAD), 0, 1).tolist()

In [ ]:
tokenized_seq = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("def max(a,b): if a>b: return a else return b"))
padded_seq = pad_max_len(tokenized_seq)
assert MAX_LEN == len(padded_seq)

In [ ]:
def tokenize_sequence(code: str):
    
    code_tokens = tokenizer.tokenize(code)
    tokens = [tokenizer.cls_token] + [tokenizer.sep_token] + code_tokens + [tokenizer.sep_token]
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    if len(tokens_ids) >= MAX_LEN:
        tokens_ids = tokens_ids[:MAX_LEN]
    
    padded_token_ids = pad_max_len(tokens_ids)
    mask_ids = get_input_mask(padded_token_ids)
    return padded_token_ids, mask_ids


In [ ]:
import tensorflow as tf
from typing import List


def embed_multiple(codes: List[str]):
    
    embeddings = []
    with torch.no_grad():
      for code in codes:
          tok_ids, att_mask = tokenize_sequence(code)
          context_embeddings = model(input_ids=torch.Tensor(tok_ids)[None, :].long().to(device),\
                            attention_mask=torch.Tensor(att_mask)[None, :].to(device)).pooler_output  #  [0] refers to last_hidden_states
          # print("context_embeddings: ", context_embeddings.pooler_output.shape)
          if not embeddings:
              embeddings.append(context_embeddings)#[:,0, :])
          else:
              embeddings.append(context_embeddings) #[:,0, :])
              emb_mean = torch.sum(torch.stack(embeddings),  axis=0)
              embeddings = [emb_mean]
      # print(embeddings)
      if len(embeddings) == 1:
          return torch.Tensor(embeddings[0])
    return torch.squeeze(embeddings[0], axis=0)


In [ ]:
embds = embed_multiple(['def hi(): print("hi")'])#, 'def hi_hello(self): print("hi morninng")'])

# torch.vstack(embds).shape
# embds

In [ ]:
# xx.pooler_output.shape

In [ ]:
# xx.last_hidden_state.shape

In [ ]:
embds.shape

torch.Size([1, 768])

In [ ]:
# # np.array(tokenize_sequence('def hi(): print("hi")')[0]).shape

# tids, attn = tokenize_sequence('def hi(): print("hi")')
# with torch.no_grad():
#   emb = model(**{"input_ids":torch.Tensor(tids)[None, :].long().to(device), "attention_mask": torch.Tensor(attn)[None, :].long().to(device)})

In [ ]:
# emb

In [ ]:
# tids

In [ ]:
4096 * 768

3145728

In [ ]:
# from transformers import AutoTokenizer, BigBirdModel
# import torch

# tokenizer_pretrained = AutoTokenizer.from_pretrained("google/bigbird-roberta-base")
# # model_pretrained = BigBirdModel.from_pretrained("google/bigbird-roberta-base")

# inputs = tokenizer_pretrained("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)
# outputs
# # last_hidden_states = outputs.last_hidden_state
# # last_hidden_states.shape

In [ ]:
# inputs.input_ids.shape

In [ ]:
# np.array(tokenize_sequence('def hi(): print("hi")')[0]).shape

In [ ]:
def find_src_path(android_dir):
    for root, dirs, files in os.walk(android_dir):
        if (root.endswith('src')):
            return root

In [ ]:
assert "./coach_repos_zip/2048-android-master/src" == find_src_path('./coach_repos_zip/2048-android-master')

In [ ]:
def find_files_recursively(src_dir: str):
    """find all .java files recursively from src dir"""
    java_files = []
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith('.java'):
                java_files.append(root + '/' + file)
    return java_files

In [ ]:
from pathlib import Path

def get_java_paths(base = BASE) -> pd.DataFrame:
    all_dirs = []
    all_java_paths = []
    for android_dir in os.listdir(base):
        src = find_src_path(base / android_dir)
        if src:
            java_paths = find_files_recursively(src)
            all_dirs.extend(len(java_paths) * [android_dir])
            all_java_paths.extend(java_paths)
    df = pd.DataFrame({"projects": all_dirs, "java_paths": all_java_paths})
    return df

In [ ]:
df = get_java_paths()
df

,projects,java_paths
0,android-mvvm-architecture-master,coach_repos_zip/android-mvvm-architecture-mast...
1,android-mvvm-architecture-master,coach_repos_zip/android-mvvm-architecture-mast...
2,android-mvvm-architecture-master,coach_repos_zip/android-mvvm-architecture-mast...
3,android-mvvm-architecture-master,coach_repos_zip/android-mvvm-architecture-mast...
4,android-mvvm-architecture-master,coach_repos_zip/android-mvvm-architecture-mast...
...,...,...
2478,BipolAlarm-master,coach_repos_zip/BipolAlarm-master/app/src/main...
2479,BipolAlarm-master,coach_repos_zip/BipolAlarm-master/app/src/main...
2480,BipolAlarm-master,coach_repos_zip/BipolAlarm-master/app/src/main...
2481,BipolAlarm-master,coach_repos_zip/BipolAlarm-master/app/src/main...


In [ ]:
df.groupby('projects').count()

,java_paths
projects,
2048-android-master,1
ANNO1404-Warenrechner-App-master,54
ATimeTracker-master,11
Android-CleanArchitecture-master-master,36
Android-MGKEapp-master,1
...,...
dotdash-keyboard-android-master,5
freeotp-android-master,20
mvvm-sample-app-master,23


In [ ]:
def read_file(path):
    with open(path) as f:
        return f.readlines()

In [ ]:
from tqdm import tqdm

i = 0

proj_embeddings = {}

df_group_by_projects = df.groupby('projects')

for x, v in tqdm(df_group_by_projects, total=len(df_group_by_projects)):
    print(x)
    i += 1 
    codes_all = []
    for file_path in v['java_paths'].values:
#         print("file_path: ", file_path)
        source_code_list = read_file(file_path)    
        source_code = ''.join(source_code_list)
        codes_all.append(source_code)
    
    code_embeddings = embed_multiple(codes_all)
#     print("code_embeddings:", len(code_embeddings))
#     break
    if len(code_embeddings) > 1:
        print("code_emb_len: ", len(code_embeddings))
#         emb_stacked = 
        # emb_mean = torch.sum(torch.vstack(code_embeddings), keepdim=True, axis=0)
        
        proj_embeddings[x] = code_embeddings
        
#         print("emb_mean.shape: ", emb_mean.shape)
    else:
        proj_embeddings[x] = code_embeddings[0]  # [768]

  0%|          | 0/64 [00:00<?, ?it/s]

2048-android-master


  2%|▏         | 1/64 [00:00<00:25,  2.50it/s]

ANNO1404-Warenrechner-App-master


  3%|▎         | 2/64 [00:21<12:47, 12.37s/it]

...

 97%|█████████▋| 62/64 [15:25<00:20, 10.49s/it]

nongbeer-mvp-android-demo-master


 98%|█████████▊| 63/64 [16:02<00:18, 18.70s/it]

yubiclip-android-master


100%|██████████| 64/64 [16:05<00:00, 15.09s/it]


In [ ]:
### Read and process labels
import pandas as pd
import numpy as np

lbl_file = "ck_results_of_repos.csv"

lbl_df = pd.read_csv(lbl_file)
lbl_df['folder_name'] = lbl_df['folder'].map(lambda x: x.split('/')[1])


In [ ]:
lbl_df

,folder,pattern,folder_name
0,ck_metrics_root/2048-android-master,NONE,2048-android-master
1,ck_metrics_root/AdAway-master,MVVM,AdAway-master
2,ck_metrics_root/amdroid-master,NONE,amdroid-master
3,ck_metrics_root/android-adbm-master,NONE,android-adbm-master
4,ck_metrics_root/android-applications-info-master,NONE,android-applications-info-master
...,...,...,...
64,ck_metrics_root/Todoer-master,NONE,Todoer-master
65,ck_metrics_root/yubiclip-android-master,MVC,yubiclip-android-master
66,ck_metrics_root/zanavi-master,MVC,zanavi-master
67,ck_metrics_root/Zensuren-master,MVC,Zensuren-master


In [ ]:
proj_to_label = dict(zip(lbl_df['folder_name'].str.lower(), lbl_df['pattern']))

labels = [proj_to_label[proj.lower()]  for proj in proj_embeddings.keys()] 

np.unique(labels)

array(['MVC', 'MVP', 'MVVM', 'NONE'], dtype='<U4')

In [ ]:
lbl2idx = {lbl: i for i, lbl in enumerate(np.unique(labels))}
idx2lbl = dict(zip(lbl2idx.values(), lbl2idx.keys()))

In [ ]:
y = [lbl2idx[l] for l in labels]

In [ ]:
from collections import Counter
Counter(y)

Counter({3: 21, 0: 25, 1: 11, 2: 7})

In [ ]:
# proj_embeddings['2048-android-master'].detach().cpu().numpy().tolist()

In [ ]:
# X = tf.convert_to_tensor(list(proj_embeddings.values()))
# X.shape

In [ ]:
X = tf.convert_to_tensor([embedding.detach().cpu().numpy().tolist() for embedding in proj_embeddings.values()])
X.shape

TensorShape([64, 768])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.15, stratify=y)

## **Modelling** **Keras**

In [ ]:
embedding_vector_length = 768

In [ ]:
import numpy
import keras
from keras import Input, Model
from keras.layers import InputSpec, Layer, GlobalMaxPooling1D
# from keras.layers import InputLayer
# from keras.models import Model, Input
from keras.layers import LSTM, Dense, GRU, Bidirectional, Dropout, SpatialDropout1D
# from keras.layers import Dense, LSTM, Dropout, Embedding, SpatialDropout1D, Bidirectional, concatenate, InputSpec
# from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
# from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras import backend as K
# from keras.optimizers import adam
# from tensorflow.python.keras.optimizers import SGD


In [ ]:
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-2,
#     decay_steps=10000,
#     decay_rate=0.9)
# optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

# fix random seed for reproducibility

# lr=0.0003, bs=32, units=60   <=> 73 (I guess)
# lr=0.0003, bs=32/48(71.18%)/64(70.95%), units=120   <=> 72.35
# lr=0.0003, bs=16(71.65%)/, units=120   <=> 72.35

from keras.regularizers import l2, l1

# kernel_regularizer=l2(0.001)

accuracy_model={}

# fix random seed for reproducibility

# lr=0.0003, bs=32, units=60   <=> 73 (I guess)
# lr=0.0003, bs=32/48(71.18%)/64(70.95%), units=120   <=> 72.35
# lr=0.0003, bs=16(71.65%)/, units=120   <=> 72.35

act = 'tanh'

cvscores = []
for _ in range(5):
    
    emb_input = Input(shape=(embedding_vector_length,))

    # x = GlobalMaxPooling1D()(emb_input)

    x = Dense(256, activation=act)(emb_input)  # , kernel_regularizer=l2(0.0001)
    
    x = Dropout(0.4)(x)
    x2 = Dense(128, activation=act)(x)  #, kernel_regularizer=l2(0.002)
    # emb_input_dr = SpatialDropout1D(0.2)(emb_input)

    # bilstm_enc = Bidirectional(LSTM(units=60, return_sequences=True,
    #                               dropout=0.5,
    #                             recurrent_dropout=0.3))(emb_input)
    # x, attn = AttentionWeightedAverage(return_attention=True)(bilstm_enc)

    out = Dense(len(set(y)), activation="softmax")(x2)
    model = Model(emb_input, out)
    # opt = tf.keras.optimizers.Adam(lr=0.001)
    opt  = tf.keras.optimizers.legacy.SGD(lr=1e-4, momentum=0.5, decay=0.01, nesterov=False)

#     opt_rms = tf.keras.optimizers.RMSprop(lr=0.001) 
    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=['accuracy'])  # 'binary_crossentropy'
    
    print(model.summary())
    history = model.fit(np.array(X_train),
                      np.array(y_train),
                      batch_size=16, epochs=30,
#                       class_weight=class_weights,
                      # validation_split=0.1,
#                       callbacks=[ValAccuracyManagement()],  # keras.callbacks.EarlyStopping(patience=15)
                      validation_data=(X_test,y_test),
                      verbose=1)  #.reshape(len(y_tr), max_len, 1),
    # validation_split=0.10,
    scores = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
    
#     y_label_pred = get_preds(model, X_test)
#     f1 = get_f1(y_test, y_label_pred) * 100

#     print('Testing F1 score: {}'.format(f1))
    print("Eval %s on test: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#     cvscores.append((scores[1] * 100, model, f1))
    cvscores.append((scores[1] * 100, model))
    # accuracy_model[scores[1] * 100] = 
    # break

    # 0.0005 71.83%

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 768)]             0         
                                                                 
 dense_30 (Dense)            (None, 256)               196864    
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_31 (Dense)            (None, 128)               32896     
                                                                 
 dense_32 (Dense)            (None, 4)                 516       
                                                                 
Total params: 230,276
Trainable params: 230,276
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


4/4 [==============================] - 1s 64ms/step - loss: 1.5843 - accuracy: 0.2593 - val_loss: 1.4681 - val_accuracy: 0.4000
Epoch 2/30
4/4 [==============================] - 0s 12ms/step - loss: 1.5908 - accuracy: 0.2593 - val_loss: 1.4324 - val_accuracy: 0.4000
Epoch 3/30
4/4 [==============================] - 0s 13ms/step - loss: 1.3943 - accuracy: 0.3889 - val_loss: 1.4098 - val_accuracy: 0.4000
Epoch 4/30
4/4 [==============================] - 0s 15ms/step - loss: 1.6298 - accuracy: 0.1667 - val_loss: 1.3976 - val_accuracy: 0.4000
Epoch 5/30
4/4 [==============================] - 0s 15ms/step - loss: 1.4321 - accuracy: 0.3148 - val_loss: 1.4001 - val_accuracy: 0.4000
Epoch 6/30
4/4 [==============================] - 0s 11ms/step - loss: 1.5012 - accuracy: 0.3519 - val_loss: 1.3750 - val_accuracy: 0.4000
Epoch 7/30
4/4 [==============================] - 0s 12ms/step - loss: 1.5685 - accuracy: 0.2593 - val_loss: 1.3754 - val_accuracy: 0.4000
Epoch 8/30
4/4 [======================

In [ ]:
cvscores

[(40.00000059604645, <keras.engine.functional.Functional at 0x7f5538377b20>),
 (40.00000059604645, <keras.engine.functional.Functional at 0x7f553824ad60>),
 (50.0, <keras.engine.functional.Functional at 0x7f55380c3c70>),
 (10.000000149011612, <keras.engine.functional.Functional at 0x7f5475fe2cd0>),
 (50.0, <keras.engine.functional.Functional at 0x7f5475ef8730>)]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(30)
X_pca = pca.fit_transform(X)
pca.explained_variance_ratio_.sum()

0.9999991250250353

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import balanced_accuracy_score

# def get_results(X, y, classifier, N=10):
#   """train classifier on N splits and returns result"""
#   # df_sel = df_sel.reset_index(drop=True)

#   # X, y = df_sel.drop(TARGET, axis=1), df_sel[TARGET]

#   skf = StratifiedKFold(n_splits=N)
#   skf.get_n_splits(X, y)

#   scores = []

#   for train_index, test_index in skf.split(X, y):

#       X_train, X_test = X.loc[train_index], X.loc[test_index]
#       y_train, y_test = y[train_index], y[test_index]
      
#       lr = classifier() 
#       lr.fit(X, y)

#       y_preds = lr.predict(X_test)
      
#       bal_score = balanced_accuracy_score(y_test, y_preds)    
#       scores.append(bal_score)
#   return np.mean(scores)

# def get_results(X_pca, y, classifier):
  
#   X_train, X_test, y_train, y_test = train_test_split(np.array(X_pca), np.array(y), test_size=0.15, stratify=y) #, random_state=3)
  
#   lr = classifier() 
#   lr.fit(X_train, y_train)

#   y_preds = lr.predict(X_test)

#   clf_report = classification_report(y_test, y_preds)
#   print(clf_report)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score 
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import numpy as np



def statified_results(X, y, classifier, N=5):

    skf = StratifiedKFold(n_splits=N)
    skf.get_n_splits(X, y)

    f1_scores_classwise = [] 

    for train_index, test_index in skf.split(X, y):

        X_train, X_test = X[train_index, :], X[test_index, :]
        y_train, y_test = y[train_index], y[test_index]
            
        clf = classifier() 
        clf.fit(X_train, y_train)
        
        y_preds = clf.predict(X_test)

        _, _, f, s_ = precision_recall_fscore_support(y_test, y_preds)
        f1_scores_classwise.append(f.tolist())
    return  np.mean(f1_scores_classwise, axis=0), np.std(f1_scores_classwise, axis=0)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

def xgb():
  return GradientBoostingClassifier()

In [ ]:
from sklearn.linear_model import LogisticRegression

def logistic_train():
  return LogisticRegression(solver='liblinear')

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def dt():
  return DecisionTreeClassifier()

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def lda():
  return QuadraticDiscriminantAnalysis()

In [ ]:
from sklearn.svm import SVC

def svm():
  return SVC(kernel='poly', degree=5)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def rf():
  return RandomForestClassifier()

In [ ]:
y = np.array(y)

In [ ]:
statified_results(X_pca, y, xgb)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.47272727, 0.2       , 0.        , 0.53593074]),
 array([0.05454545, 0.24494897, 0.        , 0.2337584 ]))

In [ ]:
statified_results(X_pca, y, logistic_train)

(array([0.64539683, 0.38      , 0.28      , 0.68428571]),
 array([0.13297609, 0.22070594, 0.23151674, 0.12473646]))

In [ ]:
lbl2idx

{'MVC': 0, 'MVP': 1, 'MVVM': 2, 'NONE': 3}

In [ ]:
statified_results(X_pca, y, dt)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.36161616, 0.29333333, 0.        , 0.44718615]),
 array([0.19795506, 0.1496663 , 0.        , 0.11568733]))

In [ ]:
statified_results(X_pca, y, lda)

/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_class

(array([0.38430014, 0.23333333, 0.        , 0.21587302]),
 array([0.17903833, 0.29059326, 0.        , 0.21191517]))

In [ ]:
# from sklearn import svm

def svm_nl():
  return svm.NuSVC(gamma="auto")

statified_results(X_pca, y, svm)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

(array([0.55163399, 0.        , 0.        , 0.        ]),
 array([0.0430778, 0.       , 0.       , 0.       ]))

In [ ]:
X_pca.shape

(64, 30)

In [ ]:
statified_results(X_pca, y, rf)  # pca=30

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

(array([0.61208791, 0.23333333, 0.        , 0.60761905]),
 array([0.11864061, 0.29059326, 0.        , 0.19492076]))

In [ ]:
len(y) , len(lbl_df)

(64, 69)

In [ ]:
cvscores

[(40.00000059604645, <keras.engine.functional.Functional at 0x7f5538377b20>),
 (40.00000059604645, <keras.engine.functional.Functional at 0x7f553824ad60>),
 (50.0, <keras.engine.functional.Functional at 0x7f55380c3c70>),
 (10.000000149011612, <keras.engine.functional.Functional at 0x7f5475fe2cd0>),
 (50.0, <keras.engine.functional.Functional at 0x7f5475ef8730>)]

In [ ]:
# 256, 0.4, 64, Adam(0.0005), tanh
# 

In [ ]:
cvscores

[(40.00000059604645, <keras.engine.functional.Functional at 0x7f5538377b20>),
 (40.00000059604645, <keras.engine.functional.Functional at 0x7f553824ad60>),
 (50.0, <keras.engine.functional.Functional at 0x7f55380c3c70>),
 (10.000000149011612, <keras.engine.functional.Functional at 0x7f5475fe2cd0>),
 (50.0, <keras.engine.functional.Functional at 0x7f5475ef8730>)]

# **Evaluation**

In [ ]:
preds = np.argmax(cvscores[0][1].predict(X_test), -1)
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

1/1 [==============================] - 0s 51ms/step
              precision    recall  f1-score   support

           0       0.40      1.00      0.57         4
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         3

    accuracy                           0.40        10
   macro avg       0.10      0.25      0.14        10
weighted avg       0.16      0.40      0.23        10



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
preds = np.argmax(cvscores[-1][1].predict(X_test), -1)
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

1/1 [==============================] - 0s 52ms/step
              precision    recall  f1-score   support

           0       0.57      1.00      0.73         4
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.50      0.33      0.40         3

    accuracy                           0.50        10
   macro avg       0.27      0.33      0.28        10
weighted avg       0.38      0.50      0.41        10



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
